In [1]:
import Cython
%load_ext Cython

In [2]:
import time
import numpy as np
from tqdm import tqdm_notebook as tqdm

# WordEmbeding [CountVectorizer]
В данном пункте сравним скорости работы простого векторного представления предложений. В качестве базового решения выбран ```CountVectorizer``` из пакета ```sklearn```.

На основе ```CountVectorizer``` был написан сообвественый класс ```Vectorizer``` на python. Данный класс повторяет базовые возможности класса ```CountVectorizer```, такие как ```fit``` и ```transform```.

Сревнение производительности проводится для класса ```Vectorizer``` который работает в следующих случаях:
* Простая реализация на pypy3;
* Простая реалзиция на python без компиляции;
* Простая компиляция класса без использования типизации;
* Компиляция класса ```Vectorizer``` с использованием типизации данных.

## Data

In [10]:
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
with open('../data/ru.tok', 'r') as f:
    corpus = f.read().lower().splitlines()

## Models

### Python Vectorizer
Данная модель ```Vectorizer``` построенна для повторения базовой функциональности ```CountVectorizer```. В качестве поиска токенов используется такое же регулярное выражение как и в случае дефолтного запуска ```CountVectorizer```.

In [12]:
token_pattern = re.compile(r"(?u)\b\w\w+\b")
class Vectorizer:
    def __init__(self):
        self.WordToInt = dict()
        pass
    
    def split_sent(self, sent):
        return token_pattern.findall(sent)
    
    def _fit(self, corpus):
        for sent in corpus:
            words = self.split_sent(sent)
            for word in words:
                if word not in self.WordToInt:
                    self.WordToInt[word] = len(self.WordToInt)
        return
    
    def fit(self, corpus):
        return self._fit(corpus)
    
    def get_feature_names(self):
        return list(self.WordToInt.keys())
    
    def _transform(self, corpus):
        transformed = np.zeros([len(corpus), len(self.WordToInt)], dtype=np.int64)
        for i, sent in enumerate(corpus):
            words = self.split_sent(sent)
            row = transformed[i]
            LocalVocab = dict()
            for word in words:
                if word in self.WordToInt:
                    ind = self.WordToInt[word]
                    if ind not in LocalVocab:
                        LocalVocab[ind] = 1
                    else:
                        LocalVocab[ind] += 1
                    
            for ind in LocalVocab:
                row[ind]=LocalVocab[ind];
                    
        return transformed
    
    def transform(self, corpus):
        return self._transform(corpus)
    
    def fit_transform(self, corpus):
        self._fit(corpus)
        return self._transform(corpus)

### Cython Vectorizer and Cython+Typed Vectorizer

In [13]:
%%cython
import re
import numpy as np
cimport numpy as np
np.import_array()

token_pattern = re.compile(r"(?u)\b\w\w+\b")

class CythonVectorizer:
    def __init__(self):
        self.WordToInt = dict()
        pass
    
    def split_sent(self, sent):
        return token_pattern.findall(sent)
    
    def fit(self, corpus):
        for sent in corpus:
            words = self.split_sent(sent)
            for word in words:
                if word not in self.WordToInt:
                    self.WordToInt[word] = len(self.WordToInt)
        return
    
    def get_feature_names(self):
        return list(self.WordToInt.keys())
    
    def transform(self, corpus):
        transformed = np.zeros([len(corpus), len(self.WordToInt)], dtype=np.int64)
        for i, sent in enumerate(corpus):
            words = self.split_sent(sent)
            row = transformed[i]
            LocalVocab = dict()
            for word in words:
                if word in self.WordToInt:
                    ind = self.WordToInt[word]
                    if ind not in LocalVocab:
                        LocalVocab[ind] = 1
                    else:
                        LocalVocab[ind] += 1
                    
            for ind in LocalVocab:
                row[ind]=LocalVocab[ind];
                    
        return transformed
    
    def fit_transform(self, corpus):
        self._fit(corpus)
        return self._transform(corpus)




cdef class CythonTypedVectorizer:
    cdef dict WordToInt
    def __cinit__(self):
        self.WordToInt = dict()
        pass
        
    cdef list split_sent(self, str sent):
        return token_pattern.findall(sent)
    
    cpdef void fit(self, list corpus):
        cdef str word, sent
        cdef list words
        for sent in corpus:
            words = self.split_sent(sent)
            for word in words:
                if word not in self.WordToInt:
                    self.WordToInt[word] = len(self.WordToInt)
        return
    
    def get_feature_names(self):
        return list(self.WordToInt.keys())
    
    cpdef np.ndarray transform(self, list corpus):
        cdef np.ndarray row
        cdef dict LocalVocab
        cdef str word, sent
        cdef int i, ind
        cdef list words
        cdef np.ndarray transformed = np.zeros([len(corpus), len(self.WordToInt)], dtype=np.int64)
        for i, sent in enumerate(corpus):
            words = self.split_sent(sent)
            row = transformed[i]
            LocalVocab = dict()
            for word in words:
                if word in self.WordToInt:
                    ind = self.WordToInt[word]
                    if ind not in LocalVocab:
                        LocalVocab[ind] = 1
                    else:
                        LocalVocab[ind] += 1
                    
            for ind in LocalVocab:
                row[ind]=LocalVocab[ind];
                    
        return transformed
    
    def fit_transform(self, list corpus):
        self._fit(corpus)
        return self._transform(corpus)

## Comparison
В эксперименте сравнивается время метода ```fit```, а также метода ```transform``` для всех моделей. Для оценки времени производится усреднение по нескольким независимым вызовам данных методов. Каждый раз вызов метода ```fit``` производится на новом объекте рассматриваемого класса. Метод ```transform``` вызывается также каждый раз на новом объекте рассматриваемого класса после вызова метода ```fit```.

In [14]:
number_of_experiment = 200

#### Sklearn
Стандартная реализация ```CountVectorizer``` с дефолтными параметрами из пакета ```sklearn```.

In [15]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = CountVectorizer()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus).toarray()
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

print('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))


(fit=1.3924531519412995, transform=2.5378193056583402)


#### Python Vectorizer
Реализация дефолтного ```CountVectorizer``` на python без компиляциии кода.

In [16]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = Vectorizer()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus)
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

print('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))


(fit=0.6331940710544586, transform=2.1211783683300016)


#### Cython Vectorizer without Typing
Реализация дефолтного ```CountVectorizer``` на python с дальнешей cython компиляцией кода.

In [17]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = CythonVectorizer()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus)
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

print('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))


(fit=0.5791377282142639, transform=1.964157031774521)


#### Cython Vectorizer with Typing
Реализация дефолтного ```CountVectorizer``` на cython с использованием типиизации объектов с дальнешей cython компиляцией кода.

In [18]:
list_of_fit_times = []
list_of_transform_times = []
############################################

iterable = tqdm(range(number_of_experiment))
for _ in iterable:
    model = CythonTypedVectorizer()
    
    start = time.time()
    model.fit(corpus)
    end = time.time()
    
    list_of_fit_times.append(end-start)
    
    start = time.time()
    _ = model.transform(corpus)
    end = time.time()
    
    list_of_transform_times.append(end-start)
    
    iterable.set_postfix_str('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))

print('(fit={}, transform={})'.format(np.mean(list_of_fit_times), np.mean(list_of_transform_times)))


(fit=0.5451541757583618, transform=1.9546177613735198)


## Result

Оценка времени выполнялась для выборки, которая состояла из $79582$ строк ($16.8$mb текста). Всего в данном тексте содержится $76777$ различных токенов, которые были найдены всемы моделями и добавлены в словарь.


Для чистоты эксперимента время предаставленное в таблице является устредненным по $200$ вызовам функции ```fit``` и $200$ вызовам функции ```transform```.

| Функция  | Время fit | Время transform |
| ------------- | ------------- | ------------- |
| CountVectorizer  | 1393 ms  | 2538 ms |
| Vectorizer  | 633 ms | 2121 ms |
| CythonVectorizer  | 579 ms | 1964 ms |
| CythonTypedVectorizer  | 545 ms | 1955 ms |

